<a href="https://colab.research.google.com/github/AlexeyBelowzero/Alex_DA_rep/blob/main/Briefly_BCS_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb as db

Link to Dataset (Google Drive): https://drive.google.com/file/d/1Vnn7m5UQQfNIHKl-c2ZI0t78f1lXvHhr/view?usp=sharing

Link to Dataset (Kaggle): https://www.kaggle.com/datasets/shivamb/bank-customer-segmentation

In [2]:
!gdown --id 1Vnn7m5UQQfNIHKl-c2ZI0t78f1lXvHhr

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Vnn7m5UQQfNIHKl-c2ZI0t78f1lXvHhr
To: /content/BCS.zip
100% 25.4M/25.4M [00:00<00:00, 53.0MB/s]


In [3]:
!unzip /content/BCS.zip

Archive:  /content/BCS.zip
  inflating: bank_transactions.csv   


#Reading and transforming data

In [4]:
df = pd.read_csv('/content/bank_transactions.csv')

In [5]:
pd.set_option('display.max_rows', 100)
df.head(10)

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.00
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.00
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.00
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.00
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.50
5,T6,C1536588,8/10/72,F,ITANAGAR,53609.20,2/8/16,173940,676.00
6,T7,C7126560,26/1/92,F,MUMBAI,973.46,2/8/16,173806,566.00
7,T8,C1220223,27/1/82,M,MUMBAI,95075.54,2/8/16,170537,148.00
8,T9,C8536061,19/4/88,F,GURGAON,14906.96,2/8/16,192825,833.00
9,T10,C6638934,22/6/84,M,MUMBAI,4279.22,2/8/16,192446,289.11


In [6]:
df.dtypes

TransactionID               object
CustomerID                  object
CustomerDOB                 object
CustGender                  object
CustLocation                object
CustAccountBalance         float64
TransactionDate             object
TransactionTime              int64
TransactionAmount (INR)    float64
dtype: object

In [7]:
df = df.rename(columns={'TransactionAmount (INR)': 'TransactionAmount_INR'})

In [8]:
from datetime import datetime

def parse_date(date_str):
    if isinstance(date_str, str):
        date_parts = date_str.split('/')

        if len(date_parts) == 3:
            day, month, year = map(int, date_parts)
            cutoff_year = 10
            if 0 <= year <= cutoff_year:
                year = 2000 + year
            else:
                year = 1900 + year

            parsed_date = datetime(year, month, day).date()

            if parsed_date.year > 2016:
                return None

            return parsed_date

    return None


df['CustomerDOB'] = df['CustomerDOB'].apply(parse_date)

In [9]:
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], errors='coerce')

df['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'], errors='coerce')

In [10]:
df.dtypes

TransactionID                    object
CustomerID                       object
CustomerDOB              datetime64[ns]
CustGender                       object
CustLocation                     object
CustAccountBalance              float64
TransactionDate          datetime64[ns]
TransactionTime                   int64
TransactionAmount_INR           float64
dtype: object

In [11]:
df.isna().mean()

TransactionID            0.000000
CustomerID               0.000000
CustomerDOB              0.057923
CustGender               0.001049
CustLocation             0.000144
CustAccountBalance       0.002259
TransactionDate          0.000000
TransactionTime          0.000000
TransactionAmount_INR    0.000000
dtype: float64

In [12]:
df['TransactionTime'] = df['TransactionTime'].astype(str).str.zfill(6)

df['Transaction_DateTime'] = pd.to_datetime(df['TransactionDate'].astype(str) + df['TransactionTime'], format='%Y-%m-%d%H%M%S')

In [13]:
df.head(25)

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount_INR,Transaction_DateTime
0,T1,C5841053,1994-01-10,F,JAMSHEDPUR,17819.05,2016-02-08,143207,25.00,2016-02-08 14:32:07
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,141858,27999.00,2016-02-08 14:18:58
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,142712,459.00,2016-02-08 14:27:12
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,142714,2060.00,2016-02-08 14:27:14
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,181156,1762.50,2016-02-08 18:11:56
5,T6,C1536588,1972-10-08,F,ITANAGAR,53609.20,2016-02-08,173940,676.00,2016-02-08 17:39:40
6,T7,C7126560,1992-01-26,F,MUMBAI,973.46,2016-02-08,173806,566.00,2016-02-08 17:38:06
7,T8,C1220223,1982-01-27,M,MUMBAI,95075.54,2016-02-08,170537,148.00,2016-02-08 17:05:37
8,T9,C8536061,1988-04-19,F,GURGAON,14906.96,2016-02-08,192825,833.00,2016-02-08 19:28:25
9,T10,C6638934,1984-06-22,M,MUMBAI,4279.22,2016-02-08,192446,289.11,2016-02-08 19:24:46


In [14]:
df.dtypes

TransactionID                    object
CustomerID                       object
CustomerDOB              datetime64[ns]
CustGender                       object
CustLocation                     object
CustAccountBalance              float64
TransactionDate          datetime64[ns]
TransactionTime                  object
TransactionAmount_INR           float64
Transaction_DateTime     datetime64[ns]
dtype: object

In [15]:
gender_counts = df['CustGender'].value_counts(dropna=False).reset_index()
gender_counts.columns = ['CustGender', 'Count']
null_count = df['CustGender'].isnull().sum()

gender_counts_df = pd.DataFrame(gender_counts)
null_count_df = pd.DataFrame({'CustGender': ['NaN'], 'Count': [null_count]})

display(gender_counts_df)

,CustGender,Count
0,M,765530
1,F,281936
2,NaN,1100
3,T,1


In [16]:
df.to_csv('BCS_2.csv', index=False)